In [0]:
# Explore Event Hub streaming data structure

eventhub_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/streamingingestionsathya/eventhub/"

print("=" * 70)
print("EVENT HUB STREAMING DATA STRUCTURE")
print("=" * 70)
print(f"\nBase path: {eventhub_base_path}\n")

try:
    # List partitions (0, 1, 2, etc.)
    partitions = dbutils.fs.ls(eventhub_base_path)
    
    print(f"Found {len(partitions)} partition(s):\n")
    
    for partition in partitions[:5]:  # Show first 5 partitions
        if partition.isDir():
            partition_name = partition.name.rstrip('/')
            print(f"  📁 Partition: {partition_name}")
            
            # Sample the structure - look at year folders
            try:
                years = dbutils.fs.ls(partition.path)
                year_names = [y.name.rstrip('/') for y in years if y.isDir()]
                print(f"     Years: {', '.join(year_names[:3])}{'...' if len(year_names) > 3 else ''}")
                
                # Count total files in this partition (sample)
                file_count = 0
                for year in years[:1]:  # Check first year only
                    try:
                        # Recursively count files
                        all_items = dbutils.fs.ls(year.path)
                        for item in all_items:
                            if not item.isDir():
                                file_count += 1
                    except:
                        pass
                
                if file_count > 0:
                    print(f"     Sample file count: ~{file_count} files\n")
            except Exception as e:
                print(f"     Error exploring: {str(e)[:50]}\n")
    
    print("\n" + "=" * 70)
    print("Structure: eventhub/{partition}/{year}/{month}/{day}/{hour}/{minute}/")
    print("=" * 70)
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

In [0]:
# Find and read sample Event Hub files to understand the data format

print("=" * 70)
print("SAMPLING EVENT HUB DATA")
print("=" * 70)

try:
    # Find a sample file
    sample_path = f"{eventhub_base_path}0/2026/02/01/17/03/"
    
    print(f"\nSample path: {sample_path}\n")
    
    files = dbutils.fs.ls(sample_path)
    avro_files = [f for f in files if f.name.endswith('.avro')]
    
    if avro_files:
        print(f"Found {len(avro_files)} AVRO file(s)")
        print(f"Sample file: {avro_files[0].name}\n")
        
        # Read sample AVRO file
        print("Reading sample data...\n")
        df_sample = spark.read.format("avro").load(avro_files[0].path)
        
        print("Schema:")
        df_sample.printSchema()
        
        print(f"\nRecord count: {df_sample.count()}")
        
        print("\nSample records:")
        display(df_sample.limit(5))
        
    else:
        print("⚠️  No AVRO files found in sample path")
        print("Listing all files:")
        for f in files:
            print(f"  - {f.name}")
            
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\nTrying to find any available files...")
    
    try:
        # Try to find files recursively
        all_files = dbutils.fs.ls(eventhub_base_path + "0/2026/")
        print(f"\nFound {len(all_files)} items in 2026 folder")
        for item in all_files[:10]:
            print(f"  - {item.name}")
    except:
        pass

In [0]:
# Copy Event Hub data to Bronze layer using Auto Loader
# This approach processes files incrementally and handles new data automatically

from pyspark.sql.functions import current_timestamp, lit, col

# Configuration
eventhub_source = f"abfss://{container}@{storage_account}.dfs.core.windows.net/streamingingestionsathya/eventhub/"
eventhub_name = "eventhub_events"  # Name for this event hub stream

# Bronze destination
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{layer}/eventhub/{eventhub_name}/"
checkpoint_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/eventhub_to_bronze/{eventhub_name}/"

print("=" * 70)
print("COPYING EVENT HUB DATA TO BRONZE LAYER")
print("=" * 70)
print(f"\nSource: {eventhub_source}")
print(f"Destination: {bronze_eventhub_path}")
print(f"Checkpoint: {checkpoint_eventhub_path}\n")

try:
    # Read Event Hub data with Auto Loader
    df_eventhub = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "avro")  # Event Hub Capture uses AVRO
        .option("cloudFiles.schemaLocation", f"{checkpoint_eventhub_path}schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("recursiveFileLookup", "true")  # Process all nested folders
        .load(eventhub_source)
    )
    
    print("✓ Stream created")
    
    # Add metadata and partition by date
    from pyspark.sql.functions import to_date
    
    df_enriched = (df_eventhub
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_system", lit("eventhub"))
        .withColumn("load_date", to_date(current_timestamp()))  # Partition by load date
    )
    
    print("✓ Metadata columns added")
    
    # Write to bronze layer with partitioning
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_eventhub_path}checkpoint")
        .option("mergeSchema", "true")
        .partitionBy("load_date")  # Partition by date for efficient processing
        .trigger(availableNow=True)  # Process all available data then stop
        .start(bronze_eventhub_path)
    )
    
    print("✓ Write stream started")
    print("⏳ Processing Event Hub data...\n")
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data copied to Bronze layer!")
    print("=" * 70)
    
    # Verify
    df_bronze = spark.read.format("delta").load(bronze_eventhub_path)
    record_count = df_bronze.count()
    
    print(f"\nTotal records in bronze: {record_count:,}")
    print(f"\nSample data:")
    display(df_bronze.limit(10))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

In [0]:
# Verify Event Hub data in bronze layer

print("=" * 70)
print("EVENT HUB BRONZE LAYER VERIFICATION")
print("=" * 70)

try:
    df_bronze = spark.read.format("delta").load(bronze_eventhub_path)
    
    print(f"\n📊 Total records: {df_bronze.count():,}")
    
    # Show partitions
    if "load_date" in df_bronze.columns:
        print("\n📅 Load dates:")
        df_bronze.select("load_date").distinct().orderBy("load_date").show()
    
    # Show schema
    print("\n📋 Schema:")
    df_bronze.printSchema()
    
    # Show sample
    print("\n📄 Sample records:")
    display(df_bronze.limit(10))
    
    print("\n✅ Event Hub data is ready for processing!")
    print(f"\nBronze path: {bronze_eventhub_path}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("\nMake sure you've run the previous cell to copy data to bronze layer.")

In [0]:
# Add Event Hub to the main all_database_configs for unified processing

print("=" * 70)
print("ADDING EVENT HUB TO MAIN CONFIGURATION")
print("=" * 70)

# Add Event Hub as a new "database" in the configuration
eventhub_config = {
    "database_name": "eventhub",
    "bronze_path": f"abfss://{container}@{storage_account}.dfs.core.windows.net/{layer}/eventhub/",
    "silver_path": f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/",
    "checkpoint_path": f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/eventhub/",
    "tables": [eventhub_name]  # eventhub_events
}

# Check if already exists
existing_dbs = [config['database_name'] for config in all_database_configs]

if 'eventhub' not in existing_dbs:
    all_database_configs.append(eventhub_config)
    print("\n✅ Event Hub added to configuration!")
else:
    print("\n⚠️  Event Hub already in configuration")

print("\n" + "=" * 70)
print("UPDATED CONFIGURATION SUMMARY")
print("=" * 70)

for config in all_database_configs:
    db_name = config['database_name']
    tables = config['tables']
    print(f"\n📊 {db_name}: {len(tables)} table(s)")
    for table in tables:
        print(f"    - {table}")

total_tables = sum(len(config['tables']) for config in all_database_configs)
print(f"\n{'='*70}")
print(f"Total: {len(all_database_configs)} source(s), {total_tables} table(s)")
print(f"{'='*70}")

print("\n✅ Now you can run the main production pipeline (Cell 15) to process ALL data!")
print("   It will process: MySQL databases + Event Hub data")

In [0]:
# ============================================
# FULLY DYNAMIC CONFIGURATION
# Auto-discovers databases AND tables
# ============================================

# Base storage account and container
storage_account = "datamigrationsathya"
container = "datalake"

# Path structure configuration
layer = "bronze"  # bronze, silver, gold
source_system = "mysql"  # mysql, postgres, etc.

# Base path for the source system
source_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{layer}/{source_system}/"

print("=" * 70)
print("AUTO-DISCOVERING DATABASES AND TABLES")
print("=" * 70)
print(f"\nSource system: {source_system}")
print(f"Base path: {source_base_path}")

# ============================================
# AUTO-DISCOVER ALL DATABASES
# ============================================

all_database_configs = []

try:
    # List all database folders under bronze/mysql/
    database_folders = dbutils.fs.ls(source_base_path)
    
    print(f"\nFound {len(database_folders)} database(s):\n")
    
    for db_folder in database_folders:
        if db_folder.isDir():
            database_name = db_folder.name.rstrip('/')
            
            # Construct paths for this database
            bronze_path = f"{source_base_path}{database_name}/"
            silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source_system}/{database_name}/"
            checkpoint_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/{source_system}/{database_name}/"
            
            # Auto-discover tables in this database
            try:
                table_folders = dbutils.fs.ls(bronze_path)
                tables = [t.name.rstrip('/') for t in table_folders if t.isDir()]
                
                if tables:  # Only add if tables exist
                    all_database_configs.append({
                        "database_name": database_name,
                        "bronze_path": bronze_path,
                        "silver_path": silver_path,
                        "checkpoint_path": checkpoint_path,
                        "tables": tables
                    })
                    
                    print(f"  ✓ {database_name}: {len(tables)} table(s)")
                    for table in tables:
                        print(f"      - {table}")
                    print()
                else:
                    print(f"  ⚠ {database_name}: No tables found (skipping)\n")
                    
            except Exception as e:
                print(f"  ✗ {database_name}: Error reading tables - {str(e)}\n")
                continue
    
    if not all_database_configs:
        print("\n⚠️  No databases with tables found!")
        print("Please check your bronze layer structure.")
    else:
        print("=" * 70)
        print(f"SUMMARY: {len(all_database_configs)} database(s) ready to process")
        total_tables = sum(len(config['tables']) for config in all_database_configs)
        print(f"Total tables across all databases: {total_tables}")
        print("=" * 70)
        
except Exception as e:
    print(f"\n❌ Error discovering databases: {str(e)}")
    print("\nFalling back to manual configuration...")
    
    # Fallback: Manual configuration
    database_name = "retail_db"
    bronze_base_path = f"{source_base_path}{database_name}/"
    silver_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source_system}/{database_name}/"
    checkpoint_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/{source_system}/{database_name}/"
    
    # Auto-discover tables
    try:
        folders = dbutils.fs.ls(bronze_base_path)
        tables_to_process = [folder.name.rstrip('/') for folder in folders if folder.isDir()]
        print(f"Using manual database: {database_name}")
        print(f"Found {len(tables_to_process)} tables: {tables_to_process}")
    except:
        tables_to_process = ["customer_details"]
        print(f"Using fallback table list: {tables_to_process}")
    
    # Create single database config for backward compatibility
    all_database_configs = [{
        "database_name": database_name,
        "bronze_path": bronze_base_path,
        "silver_path": silver_base_path,
        "checkpoint_path": checkpoint_base_path,
        "tables": tables_to_process
    }]

print("\n✅ Configuration complete!")
print("\nNote: Auto Loader will recursively process all files in subdirectories (e.g., load_date partitions)")

In [0]:
# Comprehensive data quality analysis for MySQL bronze layer

from pyspark.sql.functions import col, count, when, isnan, isnull, sum as spark_sum, avg, min as spark_min, max as spark_max

print("=" * 70)
print("MYSQL BRONZE LAYER - DATA QUALITY ANALYSIS")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 6 (Configuration) first!")
else:
    quality_report = []
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n📊 Analyzing: {table_name}")
            print("-" * 70)
            
            try:
                # Read bronze data
                df = spark.read.parquet(f"{bronze_path}{table_name}/")
                
                total_records = df.count()
                total_columns = len(df.columns)
                
                print(f"  Total Records: {total_records:,}")
                print(f"  Total Columns: {total_columns}")
                
                # 1. NULL ANALYSIS
                print(f"\n  📋 Null Analysis:")
                null_counts = df.select([spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).collect()[0]
                
                has_nulls = False
                for col_name in df.columns:
                    null_count = null_counts[col_name]
                    if null_count > 0:
                        null_pct = (null_count / total_records) * 100
                        print(f"    ⚠ {col_name}: {null_count:,} nulls ({null_pct:.1f}%)")
                        has_nulls = True
                
                if not has_nulls:
                    print(f"    ✓ No null values found")
                
                # 2. DUPLICATE ANALYSIS
                print(f"\n  🔄 Duplicate Analysis:")
                duplicate_count = total_records - df.dropDuplicates().count()
                if duplicate_count > 0:
                    print(f"    ⚠ Found {duplicate_count:,} duplicate records ({(duplicate_count/total_records)*100:.1f}%)")
                else:
                    print(f"    ✓ No duplicates found")
                
                # 3. DATA TYPE ANALYSIS
                print(f"\n  📝 Schema:")
                for field in df.schema.fields:
                    print(f"    - {field.name}: {field.dataType}")
                
                # 4. SAMPLE DATA
                print(f"\n  📄 Sample Records (first 3):")
                display(df.limit(3))
                
                # Store quality metrics
                quality_report.append({
                    'database': db_name,
                    'table': table_name,
                    'total_records': total_records,
                    'total_columns': total_columns,
                    'has_nulls': has_nulls,
                    'duplicate_count': duplicate_count
                })
                
            except Exception as e:
                print(f"  ❌ Error analyzing {table_name}: {str(e)[:100]}")
    
    # Summary Report
    print(f"\n\n{'='*70}")
    print("QUALITY ANALYSIS SUMMARY")
    print(f"{'='*70}")
    
    for report in quality_report:
        status = "⚠ Issues Found" if (report['has_nulls'] or report['duplicate_count'] > 0) else "✓ Clean"
        print(f"\n{report['database']}.{report['table']}: {status}")
        print(f"  Records: {report['total_records']:,}")
        if report['has_nulls']:
            print(f"  ⚠ Has null values")
        if report['duplicate_count'] > 0:
            print(f"  ⚠ Has {report['duplicate_count']:,} duplicates")

In [0]:
# Data quality analysis for Event Hub bronze layer

from pyspark.sql.functions import col, count, when, length, size

print("=" * 70)
print("EVENT HUB BRONZE LAYER - DATA QUALITY ANALYSIS")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(bronze_eventhub_path)
    
    total_records = df_eh.count()
    print(f"\nTotal Records: {total_records:,}")
    print(f"Total Columns: {len(df_eh.columns)}")
    
    # 1. NULL ANALYSIS
    print(f"\n📋 Null Analysis:")
    null_counts = df_eh.select([spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_eh.columns]).collect()[0]
    
    for col_name in df_eh.columns:
        null_count = null_counts[col_name]
        if null_count > 0:
            null_pct = (null_count / total_records) * 100
            print(f"  ⚠ {col_name}: {null_count:,} nulls ({null_pct:.1f}%)")
    
    # 2. BODY FIELD ANALYSIS (contains JSON data)
    print(f"\n📦 Body Field Analysis:")
    if 'Body' in df_eh.columns:
        # Check for empty bodies
        empty_bodies = df_eh.filter(col('Body').isNull()).count()
        print(f"  Empty bodies: {empty_bodies}")
        
        # Sample body content
        print(f"\n  Sample Body content (first record):")
        sample_body = df_eh.select('Body').limit(1).collect()[0]['Body']
        if sample_body:
            import base64
            decoded = base64.b64decode(sample_body).decode('utf-8')
            print(f"  {decoded[:200]}...")
    
    # 3. DUPLICATE ANALYSIS
    print(f"\n🔄 Duplicate Analysis:")
    duplicate_count = total_records - df_eh.dropDuplicates(['SequenceNumber', 'Offset']).count()
    if duplicate_count > 0:
        print(f"  ⚠ Found {duplicate_count:,} duplicate records")
    else:
        print(f"  ✓ No duplicates found")
    
    # 4. LOAD DATE DISTRIBUTION
    print(f"\n📅 Load Date Distribution:")
    if 'load_date' in df_eh.columns:
        df_eh.groupBy('load_date').count().orderBy('load_date').show()
    
    # 5. SCHEMA
    print(f"\n📝 Schema:")
    df_eh.printSchema()
    
    # 6. SAMPLE DATA
    print(f"\n📄 Sample Records:")
    display(df_eh.limit(3))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Make sure Event Hub data has been copied to bronze layer (run Cell 20)")

In [0]:
# Define data cleaning rules and transformations

from pyspark.sql.functions import col, trim, upper, lower, regexp_replace, to_timestamp, coalesce, lit

print("=" * 70)
print("DATA CLEANING RULES DEFINITION")
print("=" * 70)

# ============================================
# CLEANING RULES CONFIGURATION
# ============================================

cleaning_rules = {
    # Global rules applied to all tables
    'global': {
        'remove_duplicates': True,
        'trim_strings': True,
        'handle_nulls': True,
        'standardize_dates': True
    },
    
    # Table-specific rules
    'table_specific': {
        'customer_details': {
            'required_fields': ['customer_id', 'customer_name'],
            'email_validation': True,
            'dedupe_key': ['customer_id']
        },
        'orders': {
            'required_fields': ['order_id'],
            'dedupe_key': ['order_id'],
            'filter_negative_amounts': True
        },
        'users': {
            'required_fields': ['user_id', 'name'],
            'dedupe_key': ['user_id']
        },
        'eventhub_events': {
            'required_fields': ['SequenceNumber', 'Offset'],
            'dedupe_key': ['SequenceNumber', 'Offset'],
            'parse_body_json': True
        }
    }
}

print("\n✅ Cleaning rules defined:")
print(f"\n📋 Global Rules:")
for rule, enabled in cleaning_rules['global'].items():
    print(f"  - {rule}: {enabled}")

print(f"\n📋 Table-Specific Rules:")
for table, rules in cleaning_rules['table_specific'].items():
    print(f"\n  {table}:")
    for rule, value in rules.items():
        print(f"    - {rule}: {value}")

print("\n" + "=" * 70)
print("Rules ready to apply!")
print("=" * 70)

In [0]:
# Apply cleaning transformations to MySQL bronze data

from pyspark.sql.functions import col, trim, regexp_replace, when, length, coalesce

def clean_mysql_table(df, table_name, rules):
    """
    Apply cleaning transformations to a DataFrame
    """
    df_clean = df
    
    # Get table-specific rules
    table_rules = rules['table_specific'].get(table_name, {})
    
    # 1. REMOVE DUPLICATES
    if rules['global']['remove_duplicates']:
        dedupe_key = table_rules.get('dedupe_key', None)
        if dedupe_key:
            initial_count = df_clean.count()
            df_clean = df_clean.dropDuplicates(dedupe_key)
            removed = initial_count - df_clean.count()
            if removed > 0:
                print(f"    ✓ Removed {removed:,} duplicates based on {dedupe_key}")
    
    # 2. TRIM STRING COLUMNS
    if rules['global']['trim_strings']:
        string_cols = [field.name for field in df_clean.schema.fields if str(field.dataType) == 'StringType']
        for col_name in string_cols:
            df_clean = df_clean.withColumn(col_name, trim(col(col_name)))
        if string_cols:
            print(f"    ✓ Trimmed {len(string_cols)} string columns")
    
    # 3. FILTER REQUIRED FIELDS
    required_fields = table_rules.get('required_fields', [])
    if required_fields:
        initial_count = df_clean.count()
        for field in required_fields:
            if field in df_clean.columns:
                df_clean = df_clean.filter(col(field).isNotNull())
        removed = initial_count - df_clean.count()
        if removed > 0:
            print(f"    ✓ Filtered {removed:,} records with null required fields")
    
    # 4. EMAIL VALIDATION (if applicable)
    if table_rules.get('email_validation', False):
        email_cols = [c for c in df_clean.columns if 'email' in c.lower()]
        for email_col in email_cols:
            initial_count = df_clean.count()
            df_clean = df_clean.filter(
                (col(email_col).isNull()) | 
                (col(email_col).rlike(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'))
            )
            removed = initial_count - df_clean.count()
            if removed > 0:
                print(f"    ✓ Filtered {removed:,} records with invalid emails")
    
    # 5. FILTER NEGATIVE AMOUNTS (if applicable)
    if table_rules.get('filter_negative_amounts', False):
        amount_cols = [c for c in df_clean.columns if 'amount' in c.lower() or 'price' in c.lower()]
        for amount_col in amount_cols:
            if amount_col in df_clean.columns:
                initial_count = df_clean.count()
                df_clean = df_clean.filter((col(amount_col).isNull()) | (col(amount_col) >= 0))
                removed = initial_count - df_clean.count()
                if removed > 0:
                    print(f"    ✓ Filtered {removed:,} records with negative {amount_col}")
    
    return df_clean


print("=" * 70)
print("APPLYING CLEANING TRANSFORMATIONS - MYSQL")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 6 (Configuration) first!")
else:
    cleaning_summary = []
    
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n  🧹 Cleaning: {table_name}")
            
            try:
                # Read bronze data
                df_bronze = spark.read.parquet(f"{bronze_path}{table_name}/")
                initial_count = df_bronze.count()
                
                # Apply cleaning
                df_clean = clean_mysql_table(df_bronze, table_name, cleaning_rules)
                final_count = df_clean.count()
                
                removed = initial_count - final_count
                retention_pct = (final_count / initial_count) * 100 if initial_count > 0 else 0
                
                print(f"    📊 Initial: {initial_count:,} → Final: {final_count:,} ({retention_pct:.1f}% retained)")
                
                # Store cleaned DataFrame for later use
                df_clean.createOrReplaceTempView(f"cleaned_{db_name}_{table_name}")
                
                cleaning_summary.append({
                    'database': db_name,
                    'table': table_name,
                    'initial': initial_count,
                    'final': final_count,
                    'removed': removed
                })
                
            except Exception as e:
                print(f"    ❌ Error: {str(e)[:100]}")
    
    # Summary
    print(f"\n\n{'='*70}")
    print("CLEANING SUMMARY")
    print(f"{'='*70}")
    
    total_initial = sum(s['initial'] for s in cleaning_summary)
    total_final = sum(s['final'] for s in cleaning_summary)
    total_removed = sum(s['removed'] for s in cleaning_summary)
    
    for summary in cleaning_summary:
        print(f"\n{summary['database']}.{summary['table']}:")
        print(f"  Initial: {summary['initial']:,}")
        print(f"  Final: {summary['final']:,}")
        print(f"  Removed: {summary['removed']:,}")
    
    print(f"\n{'='*70}")
    print(f"TOTAL: {total_initial:,} → {total_final:,} (removed {total_removed:,})")
    print(f"{'='*70}")

In [0]:
# Apply cleaning transformations to Event Hub bronze data

from pyspark.sql.functions import col, from_json, schema_of_json, base64, unbase64
import json

print("=" * 70)
print("APPLYING CLEANING TRANSFORMATIONS - EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(bronze_eventhub_path)
    initial_count = df_eh.count()
    
    print(f"\nInitial records: {initial_count:,}")
    
    # 1. REMOVE DUPLICATES
    print(f"\n🧹 Cleaning steps:")
    df_clean = df_eh.dropDuplicates(['SequenceNumber', 'Offset'])
    removed_dupes = initial_count - df_clean.count()
    if removed_dupes > 0:
        print(f"  ✓ Removed {removed_dupes:,} duplicates")
    else:
        print(f"  ✓ No duplicates found")
    
    # 2. FILTER NULL REQUIRED FIELDS
    df_clean = df_clean.filter(
        col('SequenceNumber').isNotNull() & 
        col('Offset').isNotNull()
    )
    removed_nulls = df_eh.count() - df_clean.count() - removed_dupes
    if removed_nulls > 0:
        print(f"  ✓ Filtered {removed_nulls:,} records with null required fields")
    
    # 3. PARSE BODY JSON (if Body field exists)
    if 'Body' in df_clean.columns:
        print(f"\n  📦 Parsing Body JSON field...")
        
        # Decode base64 Body to string
        from pyspark.sql.functions import expr
        df_clean = df_clean.withColumn('body_string', expr("cast(unbase64(Body) as string)"))
        
        # Try to infer JSON schema from sample
        sample_body = df_clean.select('body_string').filter(col('body_string').isNotNull()).limit(1).collect()
        
        if sample_body and sample_body[0]['body_string']:
            try:
                # Infer schema from sample JSON
                json_schema = schema_of_json(sample_body[0]['body_string'])
                
                # Parse JSON
                df_clean = df_clean.withColumn('parsed_body', from_json(col('body_string'), json_schema))
                
                # Flatten parsed JSON fields
                if 'parsed_body' in df_clean.columns:
                    # Get all fields from parsed_body struct
                    parsed_fields = df_clean.select('parsed_body.*').columns
                    for field in parsed_fields:
                        df_clean = df_clean.withColumn(f"body_{field}", col(f"parsed_body.{field}"))
                    
                    print(f"    ✓ Parsed Body JSON into {len(parsed_fields)} fields")
            except Exception as e:
                print(f"    ⚠ Could not parse JSON: {str(e)[:100]}")
    
    final_count = df_clean.count()
    retention_pct = (final_count / initial_count) * 100 if initial_count > 0 else 0
    
    print(f"\n📊 Summary:")
    print(f"  Initial: {initial_count:,}")
    print(f"  Final: {final_count:,}")
    print(f"  Removed: {initial_count - final_count:,}")
    print(f"  Retention: {retention_pct:.1f}%")
    
    # Store cleaned DataFrame
    df_clean.createOrReplaceTempView("cleaned_eventhub_events")
    
    print(f"\n✅ Event Hub data cleaned and ready!")
    print(f"\n📄 Sample cleaned data:")
    display(df_clean.limit(5))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Make sure Event Hub data has been copied to bronze layer (run Cell 20)")

In [0]:
# Validate cleaned data quality

print("=" * 70)
print("DATA QUALITY VALIDATION - POST CLEANING")
print("=" * 70)

validation_results = []

# Validate MySQL tables
if 'all_database_configs' in dir() and all_database_configs:
    for config in all_database_configs:
        db_name = config['database_name']
        tables = config['tables']
        
        for table_name in tables:
            view_name = f"cleaned_{db_name}_{table_name}"
            
            try:
                df = spark.table(view_name)
                
                # Check for nulls in required fields
                table_rules = cleaning_rules['table_specific'].get(table_name, {})
                required_fields = table_rules.get('required_fields', [])
                
                null_in_required = False
                for field in required_fields:
                    if field in df.columns:
                        null_count = df.filter(col(field).isNull()).count()
                        if null_count > 0:
                            null_in_required = True
                            break
                
                # Check for duplicates
                dedupe_key = table_rules.get('dedupe_key', [])
                has_duplicates = False
                if dedupe_key:
                    total = df.count()
                    unique = df.dropDuplicates(dedupe_key).count()
                    has_duplicates = (total != unique)
                
                status = "✅ PASS" if (not null_in_required and not has_duplicates) else "⚠️ ISSUES"
                
                validation_results.append({
                    'table': f"{db_name}.{table_name}",
                    'status': status,
                    'null_in_required': null_in_required,
                    'has_duplicates': has_duplicates,
                    'record_count': df.count()
                })
                
            except Exception as e:
                validation_results.append({
                    'table': f"{db_name}.{table_name}",
                    'status': "❌ ERROR",
                    'error': str(e)[:50]
                })

# Validate Event Hub
try:
    df_eh = spark.table("cleaned_eventhub_events")
    
    null_in_required = df_eh.filter(
        col('SequenceNumber').isNull() | col('Offset').isNull()
    ).count() > 0
    
    has_duplicates = df_eh.count() != df_eh.dropDuplicates(['SequenceNumber', 'Offset']).count()
    
    status = "✅ PASS" if (not null_in_required and not has_duplicates) else "⚠️ ISSUES"
    
    validation_results.append({
        'table': 'eventhub.eventhub_events',
        'status': status,
        'null_in_required': null_in_required,
        'has_duplicates': has_duplicates,
        'record_count': df_eh.count()
    })
except:
    pass

# Display results
print("\n📋 Validation Results:\n")
for result in validation_results:
    print(f"{result['status']} {result['table']}")
    if 'record_count' in result:
        print(f"    Records: {result['record_count']:,}")
    if result.get('null_in_required'):
        print(f"    ⚠ Has nulls in required fields")
    if result.get('has_duplicates'):
        print(f"    ⚠ Has duplicate records")
    if 'error' in result:
        print(f"    ❌ Error: {result['error']}")
    print()

print("=" * 70)
passed = sum(1 for r in validation_results if r['status'] == "✅ PASS")
total = len(validation_results)
print(f"Validation: {passed}/{total} tables passed")
print("=" * 70)

In [0]:
# ============================================
# PRODUCTION: Write CLEANED MySQL data to Silver Layer
# This replaces the old Cell 19 - now uses cleaned data
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("MYSQL CLEANED DATA TO SILVER PIPELINE")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ ERROR: Please run Cell 6 first!")
else:
    print(f"\n✅ Found configuration for {len(all_database_configs)} database(s)\n")
    
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"{'='*70}")
        
        for table_name in tables:
            print(f"\n  📊 Processing: {db_name}/{table_name}")
            
            try:
                # Read CLEANED data from temp view
                view_name = f"cleaned_{db_name}_{table_name}"
                df_clean = spark.table(view_name)
                
                # Add processing metadata
                df_silver = (df_clean
                    .withColumn("processing_timestamp", current_timestamp())
                    .withColumn("source_table", lit(table_name))
                    .withColumn("source_database", lit(db_name))
                )
                
                # Write to silver layer (overwrite mode for cleaned data)
                df_silver.write\
                    .format("delta")\
                    .mode("overwrite")\
                    .option("mergeSchema", "true")\
                    .save(f"{silver_path}{table_name}/")
                
                record_count = df_silver.count()
                print(f"     ✅ Written {record_count:,} cleaned records to silver")
                
            except Exception as e:
                print(f"     ❌ Error: {str(e)[:100]}")
                continue
        
        print(f"\n  ✅ Completed database: {db_name}")
    
    print("\n" + "="*70)
    print("✅ MySQL Cleaned Data Pipeline Complete!")
    print("="*70)

---
# 📅 **UPDATED DAILY WORKFLOW - With Data Cleaning**

## **Complete Production Pipeline**

### **Phase 1: Configuration**
✅ **Cell 6** - Auto-discover MySQL databases and tables

---

### **Phase 2: Data Quality & Cleaning**
✅ **Cell 8** - Analyze MySQL bronze data quality  
✅ **Cell 9** - Analyze Event Hub bronze data quality  
✅ **Cell 10** - Define cleaning rules  
✅ **Cell 11** - Apply cleaning to MySQL data  
✅ **Cell 12** - Apply cleaning to Event Hub data  
✅ **Cell 13** - Validate cleaned data quality

---

### **Phase 3: Event Hub Processing**
✅ **Cell 20** - Event Hub → Bronze (copy streaming data)  
✅ **Cell 21** - Event Hub Bronze → Silver (transform)

---

### **Phase 4: MySQL to Silver (Cleaned)**
✅ **Cell 14** - MySQL Cleaned Data → Silver

---

### **Phase 5: Verification**
✅ **Cell 23** - Complete silver layer summary

---

## **🔄 Execution Flow**

```
1. Cell 6  → Configuration
    ↓
2. Cell 8-9  → Analyze Data Quality
    ↓
3. Cell 10  → Define Cleaning Rules
    ↓
4. Cell 11-12  → Apply Cleaning
    ↓
5. Cell 13  → Validate Cleaned Data
    ↓
6. Cell 20-21  → Event Hub Processing
    ↓
7. Cell 14  → Write Cleaned MySQL to Silver
    ↓
8. Cell 23  → Verify Everything
```

---

## **📊 Data Quality Results**

### **Issues Found in Bronze:**
- **customer_details**: 3 duplicates removed
- **orders**: 21 duplicates removed  
- **users**: 10 duplicates removed
- **Total**: 34 records filtered out (27% reduction)

### **Silver Layer Quality:**
- ✅ No duplicates
- ✅ No nulls in required fields
- ✅ Valid email formats
- ✅ No negative amounts
- ✅ All validations passed (8/8 tables)

---

## **🔑 Key Differences**

### **Old Workflow (Without Cleaning):**
```
Bronze (Raw) → Silver (Raw)
```
- All records copied as-is
- Duplicates included
- Invalid data in silver

### **New Workflow (With Cleaning):**
```
Bronze (Raw) → Analysis → Cleaning → Validation → Silver (Clean)
```
- Data quality analyzed
- Duplicates removed
- Invalid records filtered
- Clean data in silver

---

## **⏱️ Estimated Duration**

| Phase | Duration |
|-------|----------|
| Configuration | ~5 sec |
| Analysis | ~30 sec |
| Cleaning | ~1 min |
| Validation | ~20 sec |
| Event Hub | ~1 min |
| MySQL to Silver | ~30 sec |
| Verification | ~10 sec |
| **TOTAL** | **~4 minutes** |

---

## **📝 Cleaning Rules Applied**

### **Global Rules:**
- Remove duplicates
- Trim string fields
- Handle null values
- Standardize date formats

### **Table-Specific Rules:**
- **customer_details**: Email validation, dedupe by customer_id
- **orders**: Filter negative amounts, dedupe by order_id
- **users**: Dedupe by user_id
- **eventhub_events**: Parse JSON body, dedupe by SequenceNumber

---

## **🚀 For Databricks Workflow**

Update your workflow tasks:

**Task 1: Configuration & Cleaning**
- Run Cells: 6, 8, 9, 10, 11, 12, 13

**Task 2: Event Hub Processing**  
- Run Cells: 20, 21

**Task 3: MySQL to Silver**
- Run Cell: 14

**Task 4: Verification**
- Run Cell: 23

In [0]:
# ============================================
# OPTIONAL: Filter tables by pattern
# ============================================

# Option 1: Process only specific tables
include_tables = []  # Leave empty to process all, or specify: ["customer_details", "orders"]

# Option 2: Exclude specific tables
exclude_tables = []  # Example: ["temp_table", "test_table"]

# Option 3: Filter by prefix/pattern
table_prefix = ""  # Example: "customer_" to process only customer_* tables

# Apply filters
if include_tables:
    tables_to_process = [t for t in tables_to_process if t in include_tables]
    print(f"\nFiltered to include only: {include_tables}")

if exclude_tables:
    tables_to_process = [t for t in tables_to_process if t not in exclude_tables]
    print(f"\nExcluded tables: {exclude_tables}")

if table_prefix:
    tables_to_process = [t for t in tables_to_process if t.startswith(table_prefix)]
    print(f"\nFiltered by prefix '{table_prefix}'")

print(f"\nFinal tables to process ({len(tables_to_process)}):")
for table in tables_to_process:
    print(f"  ✓ {table}")

In [0]:
# ============================================
# MULTI-DATABASE CONFIGURATION
# Use this if you have multiple databases to process
# ============================================

# Define multiple database configurations
database_configs = [
    {
        "source_system": "mysql",
        "database_name": "retail_db",
    },
    # Uncomment and add more databases as needed:
    # {
    #     "source_system": "mysql",
    #     "database_name": "analytics_db",
    # },
    # {
    #     "source_system": "postgres",
    #     "database_name": "crm_db",
    # },
]

# Build configuration for all databases
all_configs = []

for config in database_configs:
    source = config["source_system"]
    db = config["database_name"]
    
    bronze_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{layer}/{source}/{db}/"
    silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/{source}/{db}/"
    checkpoint_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/{source}/{db}/"
    
    # Auto-discover tables for this database
    try:
        folders = dbutils.fs.ls(bronze_path)
        tables = [folder.name.rstrip('/') for folder in folders if folder.isDir()]
        
        all_configs.append({
            "source_system": source,
            "database_name": db,
            "bronze_path": bronze_path,
            "silver_path": silver_path,
            "checkpoint_path": checkpoint_path,
            "tables": tables
        })
        
        print(f"\n✓ {source}/{db}: Found {len(tables)} tables")
        
    except Exception as e:
        print(f"\n✗ {source}/{db}: Error - {str(e)}")

print(f"\n{'='*60}")
print(f"Total databases configured: {len(all_configs)}")
print(f"{'='*60}")

# Display summary
for config in all_configs:
    print(f"\n{config['source_system']}/{config['database_name']}:")
    print(f"  Tables: {', '.join(config['tables'][:5])}{'...' if len(config['tables']) > 5 else ''}")
    print(f"  Total: {len(config['tables'])} tables")

## Handling Date-Partitioned Folders

Your bronze layer has a structure like:
```
customer_details/
  ├── load_date=2026-01-31/
  │   ├── file1.parquet
  │   ├── file2.parquet
  ├── load_date=2026-02-01/
  │   ├── file1.parquet
  │   ├── file2.parquet
```

**Good news:** Auto Loader (Options 1 & 4) automatically handles this!

* **`recursiveFileLookup=true`** processes all files in all subdirectories
* **Partition columns** (like `load_date`) are automatically extracted and added as columns
* **New date folders** are automatically discovered and processed

**For batch processing** (Options 2 & 3), Spark also automatically reads partition columns when you use the wildcard pattern or specify the parent folder.

In [0]:
# Run this to verify that partition columns (like load_date) are automatically captured

table_name = "customer_details"

# Read with partition discovery
df_with_partitions = spark.read.parquet(f"{bronze_base_path}{table_name}/")

print("Schema with partition columns:")
df_with_partitions.printSchema()

print("\nSample data showing load_date column:")
display(df_with_partitions.select("*", "load_date").limit(10))

print("\nDistinct load dates in the data:")
df_with_partitions.select("load_date").distinct().show()

In [0]:
# Process a single table using Auto Loader (cloudFiles)
# Auto Loader automatically discovers new files and processes them incrementally

table_name = "customer_details"

# Read all files from all subdirectories using Auto Loader
df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")  # Change to "json", "csv", "avro" as needed
    .option("cloudFiles.schemaLocation", f"{checkpoint_base_path}{table_name}/schema")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("recursiveFileLookup", "true")  # Process files in all subdirectories
    .load(f"{bronze_base_path}{table_name}/")
)

# Add metadata columns
from pyspark.sql.functions import current_timestamp, input_file_name

df_enriched = (df
    .withColumn("processing_timestamp", current_timestamp())
    .withColumn("source_file", input_file_name())
)

# Display sample
display(df_enriched.limit(10))

In [0]:
# Alternative: Read all parquet files across all folders at once
# This is simpler but doesn't track which files have been processed

# Use wildcard to read all parquet files in all subdirectories
df_all = spark.read.parquet(f"{bronze_base_path}*/**/")

print(f"Total records: {df_all.count()}")
print(f"Schema:")
df_all.printSchema()

display(df_all.limit(10))

In [0]:
# Process multiple tables in a loop
# This approach processes each table separately

for table_name in tables_to_process:
    print(f"\nProcessing table: {table_name}")
    
    try:
        # Read data from bronze layer
        df = spark.read.parquet(f"{bronze_base_path}{table_name}/")
        
        # Add processing metadata
        from pyspark.sql.functions import current_timestamp, lit
        df_processed = (df
            .withColumn("processing_timestamp", current_timestamp())
            .withColumn("source_table", lit(table_name))
        )
        
        # Show sample
        print(f"Records in {table_name}: {df_processed.count()}")
        
        # Write to silver layer (optional - uncomment when ready)
        # df_processed.write.mode("overwrite").parquet(f"{silver_base_path}{table_name}/")
        
    except Exception as e:
        print(f"Error processing {table_name}: {str(e)}")
        continue

In [0]:
# Write the streaming data to silver layer using Auto Loader
# This creates an incremental pipeline that processes new files automatically

table_name = "customer_details"

# Read with Auto Loader
df_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{checkpoint_base_path}{table_name}/schema")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("recursiveFileLookup", "true")
    .load(f"{bronze_base_path}{table_name}/")
)

# Add metadata
from pyspark.sql.functions import current_timestamp, input_file_name
df_enriched = (df_stream
    .withColumn("processing_timestamp", current_timestamp())
    .withColumn("source_file", input_file_name())
)

# Write to silver layer
query = (df_enriched.writeStream
    .format("delta")  # Use Delta format for silver layer
    .option("checkpointLocation", f"{checkpoint_base_path}{table_name}/checkpoint")
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)  # Process all available files then stop
    .table(f"silver.retail_db.{table_name}")  # Or use .start(f"{silver_base_path}{table_name}/")
)

# Wait for completion
query.awaitTermination()
print(f"Processing complete for {table_name}")

In [0]:
# Explore your bronze layer structure
# This helps you understand what folders and files exist

print("=== Bronze Layer Structure ===")
print(f"\nBase path: {bronze_base_path}\n")

try:
    # List all folders in bronze layer
    folders = dbutils.fs.ls(bronze_base_path)
    
    for folder in folders:
        if folder.isDir():
            print(f"📁 {folder.name}")
            
            # List files in each folder (first level)
            try:
                subfolders = dbutils.fs.ls(folder.path)
                for subfolder in subfolders[:5]:  # Show first 5 items
                    print(f"   └─ {subfolder.name} ({subfolder.size} bytes)")
                if len(subfolders) > 5:
                    print(f"   └─ ... and {len(subfolders) - 5} more items")
            except:
                pass
            print()
except Exception as e:
    print(f"Error listing folders: {str(e)}")

---
## 📅 Daily Production Run Guide

### **Cells to Run Daily:**

1. **Cell 1** - Configuration (always run first)
2. **Cell 10** - Production pipeline (processes all tables incrementally)
3. **Cell 11** - Verification (optional, to check results)

---

### **How It Works:**

✅ **First Run (Day 1):**
- Processes ALL existing files in bronze layer
- Creates checkpoints for each table
- Writes data to silver layer

✅ **Subsequent Runs (Day 2, 3, 4...):**
- **Only processes NEW files** added since last run
- Skips already-processed files (based on checkpoint)
- Completes in seconds if no new data

---

### **Key Benefits:**

* 🚀 **Efficient**: Only reads new data
* 💰 **Cost-effective**: Minimal compute usage
* 🔄 **Idempotent**: Safe to run multiple times
* 📂 **Automatic**: Discovers new date partitions automatically
* ⚡ **Fast**: Completes quickly when no new data

---

### **Schedule Options:**

**Option A - Manual:** Run Cell 1 → Cell 10 daily

**Option B - Databricks Job:** 
- Create a scheduled job that runs this notebook daily
- Set schedule: Daily at specific time (e.g., 2 AM)
- Job will automatically run cells 1 and 10

**Option C - Workflow:**
- Use Databricks Workflows for orchestration
- Add dependencies if you have upstream processes

In [0]:
# This cell processes ALL tables incrementally using Auto Loader
# Run this daily - it will only process NEW files added since last run

from pyspark.sql.functions import current_timestamp, input_file_name, lit

print("=" * 60)
print("Starting Daily Bronze to Silver Pipeline")
print("=" * 60)

for table_name in tables_to_process:
    print(f"\n{'='*60}")
    print(f"Processing table: {table_name}")
    print(f"{'='*60}")
    
    try:
        # Read with Auto Loader (only processes new files)
        df_stream = (spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "parquet")
            .option("cloudFiles.schemaLocation", f"{checkpoint_base_path}{table_name}/schema")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("recursiveFileLookup", "true")
            .load(f"{bronze_base_path}{table_name}/")
        )
        
        # Add metadata columns
        df_enriched = (df_stream
            .withColumn("processing_timestamp", current_timestamp())
            .withColumn("source_file", input_file_name())
            .withColumn("source_table", lit(table_name))
        )
        
        # Write to silver layer
        query = (df_enriched.writeStream
            .format("delta")
            .option("checkpointLocation", f"{checkpoint_base_path}{table_name}/checkpoint")
            .option("mergeSchema", "true")
            .outputMode("append")
            .trigger(availableNow=True)  # Process all available files then stop
            .start(f"{silver_base_path}{table_name}/")
        )
        
        # Wait for completion
        query.awaitTermination()
        
        print(f"✅ Successfully processed {table_name}")
        
    except Exception as e:
        print(f"❌ Error processing {table_name}: {str(e)}")
        # Continue with next table even if one fails
        continue

print("\n" + "="*60)
print("Daily Pipeline Complete!")
print("="*60)

In [0]:
# ============================================
# PRODUCTION PIPELINE - MULTI-DATABASE SUPPORT
# This cell works with both single and multi-database configurations
# ============================================

from pyspark.sql.functions import current_timestamp, input_file_name, lit

print("=" * 70)
print("Starting Daily Bronze to Silver Pipeline (Multi-Database)")
print("=" * 70)

# Check if multi-database config exists, otherwise use single database
if 'all_configs' in locals() and all_configs:
    configs_to_process = all_configs
    print(f"\nProcessing {len(configs_to_process)} database(s)")
else:
    # Single database mode
    configs_to_process = [{
        "source_system": source_system,
        "database_name": database_name,
        "bronze_path": bronze_base_path,
        "silver_path": silver_base_path,
        "checkpoint_path": checkpoint_base_path,
        "tables": tables_to_process
    }]
    print(f"\nProcessing single database: {source_system}/{database_name}")

# Process each database
for config in configs_to_process:
    source = config['source_system']
    db = config['database_name']
    bronze_path = config['bronze_path']
    silver_path = config['silver_path']
    checkpoint_path = config['checkpoint_path']
    tables = config['tables']
    
    print(f"\n{'='*70}")
    print(f"Database: {source}/{db}")
    print(f"Tables to process: {len(tables)}")
    print(f"{'='*70}")
    
    # Process each table in this database
    for table_name in tables:
        print(f"\n  Processing: {source}/{db}/{table_name}")
        
        try:
            # Read with Auto Loader (only processes new files)
            df_stream = (spark.readStream
                .format("cloudFiles")
                .option("cloudFiles.format", "parquet")
                .option("cloudFiles.schemaLocation", f"{checkpoint_path}{table_name}/schema")
                .option("cloudFiles.inferColumnTypes", "true")
                .option("recursiveFileLookup", "true")
                .load(f"{bronze_path}{table_name}/")
            )
            
            # Add metadata columns
            df_enriched = (df_stream
                .withColumn("processing_timestamp", current_timestamp())
                .withColumn("source_file", input_file_name())
                .withColumn("source_table", lit(table_name))
                .withColumn("source_system", lit(source))
                .withColumn("source_database", lit(db))
            )
            
            # Write to silver layer
            query = (df_enriched.writeStream
                .format("delta")
                .option("checkpointLocation", f"{checkpoint_path}{table_name}/checkpoint")
                .option("mergeSchema", "true")
                .outputMode("append")
                .trigger(availableNow=True)
                .start(f"{silver_path}{table_name}/")
            )
            
            # Wait for completion
            query.awaitTermination()
            
            print(f"  ✅ Successfully processed {table_name}")
            
        except Exception as e:
            print(f"  ❌ Error processing {table_name}: {str(e)}")
            continue
    
    print(f"\n✅ Completed database: {source}/{db}")

print("\n" + "="*70)
print("Daily Pipeline Complete!")
print("="*70)

In [0]:
# ============================================
# DAILY PRODUCTION PIPELINE
# Processes all databases and tables discovered in Cell 1
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("Starting Daily Bronze to Silver Pipeline")
print("=" * 70)

# Use the all_database_configs from Cell 1
if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ ERROR: Please run Cell 1 first to discover databases and tables!")
    print("Cell 1 creates the 'all_database_configs' variable needed for processing.")
else:
    print(f"\n✅ Found configuration for {len(all_database_configs)} database(s)")
    total_tables = sum(len(config['tables']) for config in all_database_configs)
    print(f"Total tables to process: {total_tables}\n")
    
    # Process each database
    for config in all_database_configs:
        db_name = config['database_name']
        bronze_path = config['bronze_path']
        silver_path = config['silver_path']
        checkpoint_path = config['checkpoint_path']
        tables = config['tables']
        
        print(f"\n{'='*70}")
        print(f"Database: {db_name}")
        print(f"Tables: {len(tables)}")
        print(f"{'='*70}")
        
        # Process each table in this database
        for table_name in tables:
            print(f"\n  📊 Processing: {db_name}/{table_name}")
            
            try:
                # Read with Auto Loader (only processes new files)
                df_stream = (spark.readStream
                    .format("cloudFiles")
                    .option("cloudFiles.format", "parquet")
                    .option("cloudFiles.schemaLocation", f"{checkpoint_path}{table_name}/schema")
                    .option("cloudFiles.inferColumnTypes", "true")
                    .option("recursiveFileLookup", "true")
                    .load(f"{bronze_path}{table_name}/")
                )
                
                # Add metadata columns
                df_enriched = (df_stream
                    .withColumn("processing_timestamp", current_timestamp())
                    .withColumn("source_table", lit(table_name))
                    .withColumn("source_database", lit(db_name))
                )
                
                # Write to silver layer
                query = (df_enriched.writeStream
                    .format("delta")
                    .option("checkpointLocation", f"{checkpoint_path}{table_name}/checkpoint")
                    .option("mergeSchema", "true")
                    .outputMode("append")
                    .trigger(availableNow=True)
                    .start(f"{silver_path}{table_name}/")
                )
                
                # Wait for completion
                query.awaitTermination()
                
                print(f"     ✅ Successfully processed {table_name}")
                
            except Exception as e:
                print(f"     ❌ Error processing {table_name}: {str(e)[:100]}")
                continue
        
        print(f"\n  ✅ Completed database: {db_name}")
    
    print("\n" + "="*70)
    print("✅ Daily Pipeline Complete!")
    print("="*70)

In [0]:
# ============================================
# EVENT HUB TO BRONZE PIPELINE
# Run this to copy new Event Hub data to bronze layer
# ============================================

from pyspark.sql.functions import current_timestamp, lit, to_date

print("=" * 70)
print("EVENT HUB TO BRONZE PIPELINE")
print("=" * 70)

# Configuration
eventhub_source = f"abfss://{container}@{storage_account}.dfs.core.windows.net/streamingingestionsathya/eventhub/"
eventhub_name = "eventhub_events"

# Bronze destination
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"
checkpoint_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/eventhub_to_bronze/{eventhub_name}/"

print(f"\nSource: {eventhub_source}")
print(f"Destination: {bronze_eventhub_path}")
print(f"\n⏳ Processing Event Hub data...\n")

try:
    # Read Event Hub data with Auto Loader
    df_eventhub = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "avro")  # Event Hub Capture uses AVRO
        .option("cloudFiles.schemaLocation", f"{checkpoint_eventhub_path}schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("recursiveFileLookup", "true")
        .load(eventhub_source)
    )
    
    # Add metadata and partition by date
    df_enriched = (df_eventhub
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_system", lit("eventhub"))
        .withColumn("load_date", to_date(current_timestamp()))
    )
    
    # Write to bronze layer
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_eventhub_path}checkpoint")
        .option("mergeSchema", "true")
        .partitionBy("load_date")
        .trigger(availableNow=True)
        .start(bronze_eventhub_path)
    )
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data copied to Bronze layer!")
    print("=" * 70)
    
    # Verify
    df_bronze = spark.read.format("delta").load(bronze_eventhub_path)
    record_count = df_bronze.count()
    
    print(f"\nTotal records in bronze: {record_count:,}")
    
    if "load_date" in df_bronze.columns:
        dates = df_bronze.select("load_date").distinct().count()
        print(f"Distinct load dates: {dates}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)[:100]}")
    import traceback
    traceback.print_exc()

In [0]:
# ============================================
# EVENT HUB BRONZE TO SILVER PIPELINE
# Run this after copying Event Hub data to bronze
# ============================================

from pyspark.sql.functions import current_timestamp, lit

print("=" * 70)
print("EVENT HUB BRONZE TO SILVER PIPELINE")
print("=" * 70)

# Configuration
eventhub_name = "eventhub_events"
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"
silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"
checkpoint_silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/bronze_to_silver/eventhub/{eventhub_name}/"

print(f"\nBronze: {bronze_eventhub_path}")
print(f"Silver: {silver_eventhub_path}")
print(f"\n⏳ Processing Event Hub bronze to silver...\n")

try:
    # Read from bronze layer
    df_bronze = (spark.readStream
        .format("delta")
        .load(bronze_eventhub_path)
    )
    
    # Add processing metadata
    df_enriched = (df_bronze
        .withColumn("processing_timestamp", current_timestamp())
        .withColumn("source_table", lit(eventhub_name))
        .withColumn("source_database", lit("eventhub"))
    )
    
    # Write to silver layer
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_silver_path}checkpoint")
        .option("mergeSchema", "true")
        .trigger(availableNow=True)
        .start(silver_eventhub_path)
    )
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data processed to Silver layer!")
    print("=" * 70)
    
    # Verify
    df_silver = spark.read.format("delta").load(silver_eventhub_path)
    record_count = df_silver.count()
    
    print(f"\nTotal records in silver: {record_count:,}")
    
    if "load_date" in df_silver.columns:
        dates = df_silver.select("load_date").distinct().count()
        print(f"Distinct load dates: {dates}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)[:100]}")
    import traceback
    traceback.print_exc()

---
## 📅 **COMPLETE DAILY RUN GUIDE**

### **Daily Workflow - All Sources:**

#### **Step 1: Configuration**
✅ **Cell 6** - Auto-discover MySQL databases and tables

#### **Step 2: Event Hub Processing (Separate)**
✅ **Cell 20** - Copy new Event Hub data to Bronze layer  
✅ **Cell 21** - Process Event Hub Bronze → Silver

#### **Step 3: MySQL Processing**
✅ **Cell 19** - Process all MySQL databases Bronze → Silver

#### **Step 4: Verification (Optional)**
✅ **Cell 20 (original)** - Verify all silver layer data

---

### **Why Separate Event Hub Processing?**

* **Different source format**: Event Hub uses AVRO, MySQL uses Parquet
* **Different source location**: Event Hub from `streamingingestionsathya/`, MySQL from `bronze/mysql/`
* **Two-stage for Event Hub**: First copy to bronze, then process to silver
* **Better monitoring**: Separate cells = easier to track each source
* **Flexibility**: Run Event Hub independently if needed

---

### **Execution Order:**

```
1. Cell 6  → Discover MySQL databases/tables
2. Cell 20 → Event Hub → Bronze (copy from streaming ingestion)
3. Cell 21 → Event Hub Bronze → Silver (transform & enrich)
4. Cell 19 → MySQL Bronze → Silver (all databases)
5. Verify → Check all silver layer data
```

---

### **What Each Pipeline Does:**

**Event Hub Pipeline (Cells 20-21):**
- Reads AVRO files from `streamingingestionsathya/eventhub/`
- Copies to `bronze/eventhub/eventhub_events/`
- Processes to `silver/eventhub/eventhub_events/`
- Adds: `ingestion_timestamp`, `processing_timestamp`, `load_date`

**MySQL Pipeline (Cell 19):**
- Reads from `bronze/mysql/{database}/{table}/`
- Processes to `silver/mysql/{database}/{table}/`
- Handles: retail_db, students_db, and any new databases
- Adds: `processing_timestamp`, `source_table`, `source_database`

---

### **Scheduling Options:**

**Option A - All Together (Daily):**
- Run Cells 6, 20, 21, 19 in sequence
- Good for: Daily batch processing

**Option B - Event Hub More Frequently:**
- Event Hub (Cells 20-21): Every hour or every 15 minutes
- MySQL (Cells 6, 19): Once daily
- Good for: Near real-time Event Hub, batch MySQL

**Option C - Databricks Workflow:**
- Create workflow with tasks for each step
- Set dependencies: Config → Event Hub → MySQL → Verification
- Schedule as needed

---

### **Key Benefits:**

✅ **Unified silver layer** - All data ends up in silver layer  
✅ **Separate processing** - Event Hub and MySQL handled independently  
✅ **Incremental** - Auto Loader only processes new files  
✅ **Scalable** - Add new sources easily  
✅ **Monitored** - Each step has clear success/failure indicators

In [0]:
# Complete verification of all data sources in silver layer

print("=" * 70)
print("COMPLETE SILVER LAYER SUMMARY")
print("=" * 70)

total_sources = 0
total_tables = 0
total_records = 0

# 1. MySQL Databases
print("\n📊 MYSQL DATABASES")
print("=" * 70)

if 'all_database_configs' in dir() and all_database_configs:
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\n  Database: {db_name}")
        
        db_records = 0
        for table_name in tables:
            try:
                df = spark.read.format("delta").load(f"{silver_path}{table_name}/")
                count = df.count()
                db_records += count
                print(f"    ✓ {table_name}: {count:,} records")
            except Exception as e:
                print(f"    ✗ {table_name}: Not processed")
        
        print(f"  Subtotal: {db_records:,} records")
        total_sources += 1
        total_tables += len(tables)
        total_records += db_records
else:
    print("  ⚠ No MySQL databases configured")

# 2. Event Hub
print("\n\n📡 EVENT HUB")
print("=" * 70)

eventhub_name = "eventhub_events"
silver_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/{eventhub_name}/"

try:
    df_eh = spark.read.format("delta").load(silver_eventhub_path)
    eh_count = df_eh.count()
    
    print(f"\n  Source: eventhub")
    print(f"    ✓ {eventhub_name}: {eh_count:,} records")
    
    if "load_date" in df_eh.columns:
        dates = df_eh.select("load_date").distinct().count()
        print(f"    ✓ Load dates: {dates}")
    
    print(f"  Subtotal: {eh_count:,} records")
    total_sources += 1
    total_tables += 1
    total_records += eh_count
    
except Exception as e:
    print(f"\n  ✗ Event Hub: Not processed or error")
    print(f"     {str(e)[:80]}")

# Final Summary
print("\n\n" + "=" * 70)
print("FINAL SUMMARY")
print("=" * 70)
print(f"\n  Total Sources: {total_sources}")
print(f"  Total Tables/Streams: {total_tables}")
print(f"  Total Records: {total_records:,}")
print("\n" + "=" * 70)

if total_records > 0:
    print("\n✅ SUCCESS! All data sources processed to silver layer!")
    print("\n📍 Silver Layer Locations:")
    print(f"   - MySQL: abfss://{container}@{storage_account}.dfs.core.windows.net/silver/mysql/")
    print(f"   - Event Hub: abfss://{container}@{storage_account}.dfs.core.windows.net/silver/eventhub/")
else:
    print("\n⚠️ No data found in silver layer. Please run the processing pipelines first.")

---
# 🚀 **Databricks Workflow Setup Guide**

## **Overview**

This guide will help you create a Databricks Workflow to automate your daily data pipeline.

---

## **📋 Workflow Architecture**

```
┌─────────────────────────────────────────────────────────┐
│                    Daily Data Pipeline                   │
└─────────────────────────────────────────────────────────┘
                            │
                            ▼
              ┌─────────────────────────┐
              │  Task 1: Configuration  │
              │  (Cell 6)               │
              └─────────────────────────┘
                            │
                ┌───────────┴───────────┐
                ▼                       ▼
    ┌──────────────────────┐  ┌──────────────────────┐
    │ Task 2: Event Hub    │  │ Task 3: MySQL        │
    │ (Cells 20-21)        │  │ (Cell 19)            │
    └──────────────────────┘  └──────────────────────┘
                │                       │
                └───────────┬───────────┘
                            ▼
              ┌─────────────────────────┐
              │ Task 4: Verification    │
              │ (Cell 23)               │
              └─────────────────────────┘
```

---

## **🎯 Workflow Tasks**

### **Task 1: Configuration**
- **Cell**: 6
- **Purpose**: Auto-discover MySQL databases and tables
- **Duration**: ~5 seconds
- **Dependencies**: None

### **Task 2: Event Hub Processing**
- **Cells**: 20, 21
- **Purpose**: Copy Event Hub data to bronze, then process to silver
- **Duration**: ~1 minute
- **Dependencies**: Task 1

### **Task 3: MySQL Processing**
- **Cell**: 19
- **Purpose**: Process all MySQL databases from bronze to silver
- **Duration**: ~2 minutes
- **Dependencies**: Task 1

### **Task 4: Verification**
- **Cell**: 23
- **Purpose**: Verify all data processed successfully
- **Duration**: ~10 seconds
- **Dependencies**: Task 2, Task 3

---

## **⚙️ Setup Instructions**

Follow these steps to create the workflow in Databricks:

### **Step 1: Navigate to Workflows**
1. In Databricks workspace, click **Workflows** in the left sidebar
2. Click **Create Job** button

### **Step 2: Configure Job Settings**
1. **Name**: `Daily_Bronze_to_Silver_Pipeline`
2. **Description**: `Automated daily pipeline to process MySQL and Event Hub data from bronze to silver layer`

### **Step 3: Add Tasks**

Click **Add task** for each of the following:

#### **Task 1: Configuration**
- **Task name**: `01_Configuration`
- **Type**: Notebook
- **Source**: Workspace
- **Path**: `/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`
- **Cluster**: Use existing serverless cluster
- **Base parameters**: 
  ```json
  {
    "run_cells": "6"
  }
  ```
- **Depends on**: (none)

#### **Task 2: Event Hub Processing**
- **Task name**: `02_EventHub_Processing`
- **Type**: Notebook
- **Source**: Workspace
- **Path**: `/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`
- **Cluster**: Use existing serverless cluster
- **Base parameters**: 
  ```json
  {
    "run_cells": "20,21"
  }
  ```
- **Depends on**: `01_Configuration`

#### **Task 3: MySQL Processing**
- **Task name**: `03_MySQL_Processing`
- **Type**: Notebook
- **Source**: Workspace
- **Path**: `/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`
- **Cluster**: Use existing serverless cluster
- **Base parameters**: 
  ```json
  {
    "run_cells": "19"
  }
  ```
- **Depends on**: `01_Configuration`

#### **Task 4: Verification**
- **Task name**: `04_Verification`
- **Type**: Notebook
- **Source**: Workspace
- **Path**: `/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`
- **Cluster**: Use existing serverless cluster
- **Base parameters**: 
  ```json
  {
    "run_cells": "23"
  }
  ```
- **Depends on**: `02_EventHub_Processing`, `03_MySQL_Processing`

### **Step 4: Configure Schedule**
1. Click **Add trigger** → **Scheduled**
2. **Schedule type**: Cron
3. **Cron expression**: `0 2 * * *` (runs daily at 2 AM)
4. **Timezone**: Select your timezone
5. **Pause status**: Active

### **Step 5: Configure Alerts (Optional)**
1. Click **Add email notifications**
2. **On failure**: Add your email
3. **On success**: (optional) Add your email

### **Step 6: Save and Test**
1. Click **Create**
2. Click **Run now** to test the workflow
3. Monitor the execution in the **Runs** tab

---

## **📅 Schedule Options**

### **Daily at 2 AM**
```
0 2 * * *
```

### **Every 6 hours**
```
0 */6 * * *
```

### **Weekdays at 3 AM**
```
0 3 * * 1-5
```

### **Every hour (for Event Hub only)**
Create a separate workflow for Event Hub:
```
0 * * * *
```

---

## **🔍 Monitoring**

### **Check Workflow Status**
1. Go to **Workflows** → **Daily_Bronze_to_Silver_Pipeline**
2. Click **Runs** tab
3. View execution history and logs

### **View Task Logs**
1. Click on a specific run
2. Click on individual tasks to see logs
3. Check for errors or warnings

### **Monitor Data Quality**
- Run Cell 23 manually to check record counts
- Set up alerts for unexpected data volumes
- Monitor checkpoint locations for growth

---

## **⚠️ Important Notes**

✅ **Serverless Cluster**: The workflow uses your existing serverless cluster  
✅ **Incremental Processing**: Only new files are processed (checkpoint-based)  
✅ **Parallel Execution**: Event Hub and MySQL tasks run in parallel  
✅ **Error Handling**: If one task fails, others continue  
✅ **Idempotent**: Safe to re-run without duplicating data  

---

## **🎯 Next Steps**

1. ✅ Create the workflow using the instructions above
2. ✅ Test with "Run now"
3. ✅ Monitor first scheduled run
4. ✅ Set up email alerts
5. ✅ Document any custom configurations

In [0]:
# Alternative: Use this JSON to create the workflow via API or import

import json

workflow_config = {
    "name": "Daily_Bronze_to_Silver_Pipeline",
    "email_notifications": {
        "on_failure": ["sathyarajeshpk@gmail.com"],
        "no_alert_for_skipped_runs": False
    },
    "timeout_seconds": 0,
    "max_concurrent_runs": 1,
    "tasks": [
        {
            "task_key": "01_Configuration",
            "notebook_task": {
                "notebook_path": "/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup",
                "source": "WORKSPACE"
            },
            "existing_cluster_id": "serverless",
            "timeout_seconds": 0,
            "email_notifications": {}
        },
        {
            "task_key": "02_EventHub_Processing",
            "depends_on": [
                {
                    "task_key": "01_Configuration"
                }
            ],
            "notebook_task": {
                "notebook_path": "/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup",
                "source": "WORKSPACE"
            },
            "existing_cluster_id": "serverless",
            "timeout_seconds": 0,
            "email_notifications": {}
        },
        {
            "task_key": "03_MySQL_Processing",
            "depends_on": [
                {
                    "task_key": "01_Configuration"
                }
            ],
            "notebook_task": {
                "notebook_path": "/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup",
                "source": "WORKSPACE"
            },
            "existing_cluster_id": "serverless",
            "timeout_seconds": 0,
            "email_notifications": {}
        },
        {
            "task_key": "04_Verification",
            "depends_on": [
                {
                    "task_key": "02_EventHub_Processing"
                },
                {
                    "task_key": "03_MySQL_Processing"
                }
            ],
            "notebook_task": {
                "notebook_path": "/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup",
                "source": "WORKSPACE"
            },
            "existing_cluster_id": "serverless",
            "timeout_seconds": 0,
            "email_notifications": {}
        }
    ],
    "schedule": {
        "quartz_cron_expression": "0 0 2 * * ?",
        "timezone_id": "UTC",
        "pause_status": "UNPAUSED"
    },
    "format": "MULTI_TASK"
}

# Print formatted JSON
print("=" * 70)
print("DATABRICKS WORKFLOW JSON CONFIGURATION")
print("=" * 70)
print("\nCopy this JSON to import the workflow:")
print("\n" + json.dumps(workflow_config, indent=2))
print("\n" + "=" * 70)
print("\nTo import:")
print("1. Go to Workflows → Create Job")
print("2. Click 'JSON' tab")
print("3. Paste the JSON above")
print("4. Click 'Create'")
print("=" * 70)

---
# 📋 **Quick Reference: Daily Workflow**

## **Workflow Name**
`Daily_Bronze_to_Silver_Pipeline`

---

## **Tasks Overview**

| Task | Cell(s) | Purpose | Duration | Dependencies |
|------|---------|---------|----------|-------------|
| **01_Configuration** | 6 | Discover databases/tables | ~5 sec | None |
| **02_EventHub_Processing** | 20, 21 | Event Hub → Bronze → Silver | ~1 min | Task 1 |
| **03_MySQL_Processing** | 19 | MySQL Bronze → Silver | ~2 min | Task 1 |
| **04_Verification** | 23 | Verify all data | ~10 sec | Task 2, 3 |

---

## **Execution Flow**

```
Task 1 (Config)
    ↓
    ├─→ Task 2 (Event Hub) ─┐
    │                        ├─→ Task 4 (Verify)
    └─→ Task 3 (MySQL) ─────┘
```

---

## **Schedule**

**Default**: Daily at 2:00 AM UTC

**Cron**: `0 2 * * *`

---

## **Notebook Path**

```
/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup
```

---

## **Cells to Run**

### **Manual Run (All at once)**
```python
# Run these cells in order:
Cell 6  → Configuration
Cell 20 → Event Hub to Bronze
Cell 21 → Event Hub to Silver
Cell 19 → MySQL to Silver
Cell 23 → Verification
```

### **Workflow Run (Automated)**
- Task 1 runs Cell 6
- Task 2 runs Cells 20, 21
- Task 3 runs Cell 19
- Task 4 runs Cell 23

---

## **Monitoring URLs**

**Workflows Dashboard**:
```
https://<your-workspace>.azuredatabricks.net/?o=<workspace-id>#job/list
```

**This Workflow**:
```
https://<your-workspace>.azuredatabricks.net/?o=<workspace-id>#job/<job-id>
```

---

## **Common Issues & Solutions**

### **Issue**: Task fails with "variable not defined"
**Solution**: Ensure Task 1 (Configuration) completed successfully

### **Issue**: No new data processed
**Solution**: Normal! Auto Loader only processes new files

### **Issue**: Event Hub task takes long time
**Solution**: Check if there are many new AVRO files to process

### **Issue**: MySQL task fails on one table
**Solution**: Check cell 19 logs - it continues with other tables

---

## **Performance Tips**

✅ **Parallel Processing**: Event Hub and MySQL run simultaneously  
✅ **Incremental**: Only new files processed (fast on no new data)  
✅ **Serverless**: Auto-scales based on workload  
✅ **Checkpoints**: Track processed files automatically  

---

## **Contact & Support**

**Notebook Owner**: sathyarajeshpk@gmail.com  
**Notebook Path**: `/Repos/sathyarajeshpk@gmail.com/MigrationProject/Notebooks/Bronze_setup`  
**Workflow Name**: `Daily_Bronze_to_Silver_Pipeline`

In [0]:
# Verification: Check silver layer for all databases and tables

print("=" * 70)
print("SILVER LAYER VERIFICATION")
print("=" * 70)

if 'all_database_configs' not in dir() or not all_database_configs:
    print("\n❌ Please run Cell 1 first!")
else:
    total_processed = 0
    total_records = 0
    
    for config in all_database_configs:
        db_name = config['database_name']
        silver_path = config['silver_path']
        tables = config['tables']
        
        print(f"\n📊 Database: {db_name}")
        print("=" * 70)
        
        for table_name in tables:
            try:
                # Read from silver layer
                df = spark.read.format("delta").load(f"{silver_path}{table_name}/")
                count = df.count()
                
                # Get distinct load dates if column exists
                if "load_date" in df.columns:
                    dates = df.select("load_date").distinct().count()
                    print(f"  ✓ {table_name}: {count:,} records, {dates} load date(s)")
                else:
                    print(f"  ✓ {table_name}: {count:,} records")
                
                total_processed += 1
                total_records += count
                
            except Exception as e:
                print(f"  ✗ {table_name}: Not processed or error - {str(e)[:50]}")
        
        print()
    
    print("=" * 70)
    print(f"SUMMARY")
    print("=" * 70)
    print(f"Databases: {len(all_database_configs)}")
    print(f"Tables processed: {total_processed}")
    print(f"Total records: {total_records:,}")
    print("=" * 70)

In [0]:
# Test processing a single table to verify the setup works

from pyspark.sql.functions import current_timestamp, input_file_name, lit

# Use first database and first table for testing
test_config = all_database_configs[0]
test_db = test_config['database_name']
test_table = test_config['tables'][0]

print(f"📊 Testing with: {test_db}/{test_table}")
print("=" * 70)

bronze_path = f"{test_config['bronze_path']}{test_table}/"
silver_path = f"{test_config['silver_path']}{test_table}/"
checkpoint_path = f"{test_config['checkpoint_path']}{test_table}/"

print(f"\nBronze: {bronze_path}")
print(f"Silver: {silver_path}")
print(f"Checkpoint: {checkpoint_path}")

# Check bronze layer has data
print(f"\n🔍 Checking bronze layer...")
try:
    bronze_files = dbutils.fs.ls(bronze_path)
    print(f"   ✓ Found {len(bronze_files)} item(s) in bronze layer")
    for item in bronze_files[:3]:
        print(f"     - {item.name}")
except Exception as e:
    print(f"   ❌ Error: {str(e)}")
    raise

# Process with Auto Loader
print(f"\n⏳ Processing with Auto Loader...")

try:
    df_stream = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option("cloudFiles.schemaLocation", f"{checkpoint_path}schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("recursiveFileLookup", "true")
        .load(bronze_path)
    )
    
    print(f"   ✓ Stream created")
    
    # Add metadata
    df_enriched = (df_stream
        .withColumn("processing_timestamp", current_timestamp())
        .withColumn("source_file", input_file_name())
        .withColumn("source_table", lit(test_table))
        .withColumn("source_database", lit(test_db))
    )
    
    print(f"   ✓ Metadata columns added")
    
    # Write to silver
    query = (df_enriched.writeStream
        .format("delta")
        .option("checkpointLocation", f"{checkpoint_path}checkpoint")
        .option("mergeSchema", "true")
        .outputMode("append")
        .trigger(availableNow=True)
        .start(silver_path)
    )
    
    print(f"   ✓ Write stream started")
    print(f"   ⏳ Waiting for completion...")
    
    query.awaitTermination()
    
    print(f"\n✅ SUCCESS! Processed {test_db}/{test_table}")
    
    # Verify silver layer
    print(f"\n🔍 Verifying silver layer...")
    df_silver = spark.read.format("delta").load(silver_path)
    count = df_silver.count()
    print(f"   ✓ Silver layer has {count:,} records")
    
    print(f"\n🎉 Test successful! The pipeline is working correctly.")
    
except Exception as e:
    print(f"\n❌ ERROR: {str(e)}")
    import traceback
    traceback.print_exc()

In [0]:
# Run this after the daily pipeline to verify the results

print("=== Silver Layer Summary ===")
print()

for table_name in tables_to_process:
    try:
        df = spark.read.format("delta").load(f"{silver_base_path}{table_name}/")
        record_count = df.count()
        
        # Get distinct load dates
        load_dates = df.select("load_date").distinct().count()
        
        print(f"📊 {table_name}:")
        print(f"   Total records: {record_count:,}")
        print(f"   Distinct load dates: {load_dates}")
        print()
        
    except Exception as e:
        print(f"⚠️  {table_name}: Not yet processed or error - {str(e)}")
        print()

## ⚠️ Important: How Auto Loader Handles Modified Files

### **Default Behavior:**
Auto Loader tracks files by **path/name only**, not by content or modification time.

**Scenario:**
```
load_date=2026-01-31/data_abc.parquet (processed on Day 1)
load_date=2026-01-31/data_abc.parquet (modified on Day 2)
```

**Result:** Auto Loader will **SKIP** the modified file on Day 2 because the filename already exists in the checkpoint.

---

### **Solutions:**

**Option 1: Append-Only Pattern (Recommended)**
- Never modify existing files
- Always write NEW files with unique names (timestamps/UUIDs)
- Example: `data_2026-01-31_v1.parquet`, `data_2026-01-31_v2.parquet`

**Option 2: Use File Notifications (Azure Event Grid)**
- Enable `cloudFiles.useNotifications=true`
- Requires Azure Event Grid setup with proper IAM roles
- Tracks file modifications via storage events

**Option 3: Full Refresh Strategy**
- Delete checkpoint and reprocess all files
- Use for one-time fixes or major data corrections

**Option 4: Partition-Level Reprocessing**
- Delete specific partition data and checkpoint entries
- Reprocess only affected partitions

In [0]:
# OPTION 1: Use Azure Event Grid to detect file modifications
# This requires proper IAM roles on your storage account:
# - Storage Account Contributor
# - Storage Blob Data Contributor  
# - EventGrid EventSubscription Contributor
# - Storage Queue Data Contributor

from pyspark.sql.functions import current_timestamp, input_file_name, lit

table_name = "customer_details"

# Read with Auto Loader + File Notifications
df_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", f"{checkpoint_base_path}{table_name}/schema")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("recursiveFileLookup", "true")
    
    # Enable file notifications to detect modifications
    .option("cloudFiles.useNotifications", "true")  # Requires Event Grid setup
    .option("cloudFiles.includeExistingFiles", "true")  # Process existing files on first run
    
    .load(f"{bronze_base_path}{table_name}/")
)

# Add metadata
df_enriched = (df_stream
    .withColumn("processing_timestamp", current_timestamp())
    .withColumn("source_file", input_file_name())
)

# Write to silver layer
query = (df_enriched.writeStream
    .format("delta")
    .option("checkpointLocation", f"{checkpoint_base_path}{table_name}/checkpoint_with_notifications")
    .option("mergeSchema", "true")
    .outputMode("append")
    .trigger(availableNow=True)
    .start(f"{silver_base_path}{table_name}/")
)

query.awaitTermination()
print(f"Processing complete for {table_name}")

In [0]:
# OPTION 2: Full refresh - Use when you need to reprocess ALL files
# WARNING: This will reprocess ALL files, which can be expensive

table_name = "customer_details"

print(f"⚠️  WARNING: This will delete checkpoint and reprocess ALL files for {table_name}")
print(f"Checkpoint location: {checkpoint_base_path}{table_name}/")

# Uncomment the lines below to execute the full refresh
# print("\nDeleting checkpoint...")
# dbutils.fs.rm(f"{checkpoint_base_path}{table_name}/", recurse=True)
# print("✅ Checkpoint deleted")

# print("\nDeleting existing silver layer data...")
# dbutils.fs.rm(f"{silver_base_path}{table_name}/", recurse=True)
# print("✅ Silver layer data deleted")

# print("\nNow run Cell 10 to reprocess all files from scratch")

In [0]:
# OPTION 3: Reprocess specific partition (e.g., specific load_date)
# Use this when only one date partition has modified files

from pyspark.sql.functions import col

table_name = "customer_details"
partition_to_reprocess = "2026-01-31"  # Change this to the date you want to reprocess

print(f"Reprocessing partition: load_date={partition_to_reprocess}")

# Step 1: Delete data for this partition from silver layer
print(f"\nStep 1: Deleting partition data from silver layer...")
try:
    df_silver = spark.read.format("delta").load(f"{silver_base_path}{table_name}/")
    
    # Delete records for this partition
    df_filtered = df_silver.filter(col("load_date") != partition_to_reprocess)
    
    # Overwrite silver layer (excluding the partition to reprocess)
    df_filtered.write.format("delta").mode("overwrite").save(f"{silver_base_path}{table_name}/")
    print(f"✅ Deleted partition data from silver layer")
except Exception as e:
    print(f"⚠️  Error or no existing data: {str(e)}")

# Step 2: Delete checkpoint to force reprocessing
print(f"\nStep 2: Deleting checkpoint...")
try:
    dbutils.fs.rm(f"{checkpoint_base_path}{table_name}/", recurse=True)
    print(f"✅ Checkpoint deleted")
except Exception as e:
    print(f"⚠️  Error deleting checkpoint: {str(e)}")

print(f"\n✅ Ready to reprocess. Run Cell 10 to process all files (including modified partition)")

## ✅ Recommended Best Practice: Append-Only Pattern

### **Instead of modifying files, use unique filenames:**

**❌ Bad Pattern (causes issues):**
```
load_date=2026-01-31/data.parquet  (written on Day 1)
load_date=2026-01-31/data.parquet  (overwritten on Day 2) ← Auto Loader misses this
```

**✅ Good Pattern (works perfectly):**
```
load_date=2026-01-31/data_20260131_120000.parquet  (Day 1)
load_date=2026-01-31/data_20260131_140000.parquet  (Day 2) ← Auto Loader picks this up
```

### **Implementation Tips:**

1. **Add timestamps to filenames:**
   ```python
   from datetime import datetime
   timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
   filename = f"data_{timestamp}.parquet"
   ```

2. **Use UUIDs for uniqueness:**
   ```python
   import uuid
   filename = f"data_{uuid.uuid4()}.parquet"
   ```

3. **Configure upstream systems** to write with unique names

4. **Never overwrite existing files** in bronze layer

### **Benefits:**
- Auto Loader works perfectly
- Full audit trail of all data loads
- Easy to track data lineage
- No need for complex checkpoint management

In [0]:
# ============================================
# EVENT HUB TO BRONZE PIPELINE
# Run this to copy new Event Hub data to bronze layer
# ============================================

from pyspark.sql.functions import current_timestamp, lit, to_date

print("=" * 70)
print("EVENT HUB TO BRONZE PIPELINE")
print("=" * 70)

# Configuration
eventhub_source = f"abfss://{container}@{storage_account}.dfs.core.windows.net/streamingingestionsathya/eventhub/"
eventhub_name = "eventhub_events"

# Bronze destination
bronze_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze/eventhub/{eventhub_name}/"
checkpoint_eventhub_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoints/eventhub_to_bronze/{eventhub_name}/"

print(f"\nSource: {eventhub_source}")
print(f"Destination: {bronze_eventhub_path}")
print(f"\n⏳ Processing Event Hub data...\n")

try:
    # Read Event Hub data with Auto Loader
    df_eventhub = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "avro")  # Event Hub Capture uses AVRO
        .option("cloudFiles.schemaLocation", f"{checkpoint_eventhub_path}schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("recursiveFileLookup", "true")
        .load(eventhub_source)
    )
    
    # Add metadata and partition by date
    df_enriched = (df_eventhub
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_system", lit("eventhub"))
        .withColumn("load_date", to_date(current_timestamp()))
    )
    
    # Write to bronze layer
    query = (df_enriched.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{checkpoint_eventhub_path}checkpoint")
        .option("mergeSchema", "true")
        .partitionBy("load_date")
        .trigger(availableNow=True)
        .start(bronze_eventhub_path)
    )
    
    query.awaitTermination()
    
    print("\n" + "=" * 70)
    print("✅ Event Hub data copied to Bronze layer!")
    print("=" * 70)
    
    # Verify
    df_bronze = spark.read.format("delta").load(bronze_eventhub_path)
    record_count = df_bronze.count()
    
    print(f"\nTotal records in bronze: {record_count:,}")
    
    if "load_date" in df_bronze.columns:
        dates = df_bronze.select("load_date").distinct().count()
        print(f"Distinct load dates: {dates}")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)[:100]}")
    import traceback
    traceback.print_exc()